In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")


C:\Users\Ilya K\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pretrained_model_path = r"C:\Users\Ilya K\Desktop\chat_bot_vision\pretrained_model"
model.save_pretrained(pretrained_model_path)
tokenizer.save_pretrained(pretrained_model_path)

('C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\pretrained_model\\tokenizer_config.json',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\pretrained_model\\special_tokens_map.json',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\pretrained_model\\vocab.json',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\pretrained_model\\merges.txt',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\pretrained_model\\added_tokens.json',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\pretrained_model\\tokenizer.json')

In [2]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

In [3]:
df = pd.read_csv(r'C:\Users\Ilya K\Desktop\chat_bot_vision\new_dataset1.csv', encoding='utf-8')
df.head(10)

,question,answer
0,Кто является целевой аудиторией вашего сервиса?,Целевой аудиторией нашего сервиса являются сел...
1,Какие основные функции предоставляет ваш сервис?,Основными функциями сервиса является просмотр ...
2,Какие задачи решает ваш сервис?,Сервис решает задачи грамотного отображения ст...
3,Какие могут быть ограничения у вашего сервиса?,Доступ к сервису платный и осуществляется поср...
4,Как быстро сервис заполняет данные после регис...,После регистрации пользователя данные за после...
5,Как обеспечивается конфиденциальность данных в...,Данные хранятся на закрытой локальной базе дан...
6,Можно ли обновлять данные вручную?,"Да, у сервиса есть возможность принудительного..."
7,Какие метрики отображаются на дашборде 'Остатк...,"На дашборде отображаются остатки товаров, обор..."
8,Как рассчитывается оборачиваемость товаров?,Оборачиваемость рассчитывается как отношение к...
9,Какие визуализации используются на дашборде 'О...,На дашборде используются тепловая карта остатк...


In [4]:
df['text'] = df['question'] + " " + df['answer']
train_dataset = Dataset.from_pandas(df[['text']])

In [6]:
def tokenize_function(examples):
    result = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
    result["labels"] = result["input_ids"].copy()  
    return result


tokenized_datasets = train_dataset.map(tokenize_function, batched=True)


training_args = TrainingArguments(
    output_dir=r'C:\Users\Ilya K\Desktop\chat_bot_vision\results',
    num_train_epochs=2,
    per_device_train_batch_size=4,
    save_strategy="epoch",  
    eval_strategy="no",  
    save_total_limit=1,
    logging_dir=r'C:\Users\Ilya K\Desktop\chat_bot_vision\logs',
    logging_steps=100,
    load_best_model_at_end=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)


trainer.train()


finetuned_model_path = r'C:\Users\Ilya K\Desktop\chat_bot_vision\finetuned_model'
model.save_pretrained(finetuned_model_path)
tokenizer.save_pretrained(finetuned_model_path)

Map: 100%|██████████████████████████████████████████████████████████████████| 712/712 [00:00<00:00, 3818.23 examples/s]


Step,Training Loss
100,0.667400
200,0.061200
300,0.041700


('C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\finetuned_model\\tokenizer_config.json',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\finetuned_model\\special_tokens_map.json',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\finetuned_model\\vocab.json',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\finetuned_model\\merges.txt',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\finetuned_model\\added_tokens.json',
 'C:\\Users\\Ilya K\\Desktop\\chat_bot_vision\\finetuned_model\\tokenizer.json')